In [1]:
from navground import sim,core
import h5py
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import distance_matrix
import scipy.spatial.distance as dist
import matplotlib.colors as colors
import matplotlib.cm as cmx
import pandas as pd
from tqdm import tqdm

import matplotlib.colors as colors
import matplotlib.cm as cmx
from navground.sim.ui.video import display_video, display_video_from_run

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

2024-08-21 10:42:33.713841: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 10:42:33.717387: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 10:42:33.724433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 10:42:33.736096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 10:42:33.739289: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 10:42:33.748163: I tensorflow/core/platform/cpu_feature_gu

Las colisiones vienen dadas en un array con número de filas igual al número de colisiones en dicha simulación completa, es decir si hay 50 colisiones durante una simulacion, tendrá 50 filas. La 1 columna indica el instante de tiempo en el que se produce la colisión y las otras 2 columnas especifican el id de los robots que chocan en dicho instante.

## Example simulation

In [3]:
sideLength=4
yaml = f"""
steps: {1000}
time_step: {0.1}
record_pose: true
record_collisions: true
record_efficacy: true
scenario:
    type: Cross
    agent_margin: 0.1
    side: {sideLength}
    target_margin: 0.1
    tolerance: 0.5
    groups:
      -
        type: thymio
        number: 20
        radius: 0.08
        control_period: 0.1
        speed_tolerance: 0.02
        kinematics:
          type: 2WDiff
          wheel_axis: 0.094
          max_speed: 0.166
        behavior:
          type: HL
          optimal_speed: 0.12
          horizon: 5.0
          tau: 0.5
          eta: 0.25
          safety_margin: 0.0
        horizon: 5.0
        state_estimation:
          type: Bounded
          range: 5.0
"""

experimentHL= sim.load_experiment(yaml)
# experimentHL = sim.Experiment() # si solo especificas el scenario en el yaml esto es necesario
# experimentHL.scenario = sim.load_scenario(yaml.format(sideLength=sideLength))
# experimentHL.steps = 1000
# experimentHL.record_config.collisions = True # si no ponemos esto en el yaml no las recoge.
# experimentHL.record_config.pose = True # igual, esto es necesario para el video, recoger la posicion de los robots
experimentHL.run(number_of_threads = 0)
print(experimentHL.duration)
print(len(experimentHL.runs[0].collisions))

0:00:00.239681
81


In [4]:
print(experimentHL.time_step)
print(experimentHL.steps)

0.10000000149011612
1000


In [5]:
import matplotlib.colors as colors
import matplotlib.cm as cmx
from navground.sim.ui.video import display_video, display_video_from_run

def linear_map(a, b, cmap):
    c = cmx.ScalarMappable(norm=colors.Normalize(vmin=a, vmax=b), cmap=cmap)  
    def f(v):
        r, g, b, _ = c.to_rgba(v)
        return f"#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}"
    return f


fill_map = linear_map(0.0, 1.0, cmap=cmx.RdYlGn)

def f(entity):
    return {'fill': 'red' if entity.has_been_in_collision_since(run.world.time - 0.5) else 'lightgreen'}


run = experimentHL.runs[0]
display_video_from_run(run, factor=5.0,
            bounds=(np.array([-sideLength/1.5,-sideLength/1.5]),np.array([sideLength/1.5,sideLength/1.5])), 
             decorate=f, width=300)

# Multiple simulations

In [15]:
time_instances=1000
time_step=0.1
simulations = 1000
#cualquier espacio que falte en el yaml o algo mal escrito es error
yaml = f"""
steps: {time_instances}
time_step: {time_step}
runs: {simulations}
record_pose: true
record_twist: true
record_collisions: true
record_deadlocks: true
record_efficacy: true
run_index: 0
scenario:
  type: Cross
  agent_margin: 0.1
  side: 4
  target_margin: 0.1
  tolerance: 0.5
  groups:
    -
      type: thymio
      number: 40
      radius: 0.08
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: 0.166
      behavior:
        type: HL
        optimal_speed: 0.12
        rotation_tau: 0.5
        tau: 
          sampler: uniform
          from: 0.0
          to: 1.0
          once: true  
        eta: 
          sampler: uniform
          from: 0.0
          to: 1.0
          once: true  
        safety_margin:
          sampler: uniform
          from: 0.0
          to: 0.1
          once: true
        horizon: 5.0
      state_estimation:
        type: Bounded
        range: 5
"""

experimentHL = sim.load_experiment(yaml)
# wHL = sim.World()
# experimentHL.scenario.init_world(wHL)
experimentHL.run(number_of_threads = 12) # con 0 hilos 10 segundos 10 simulaciones, con 12 hilos 1 segundo 10 simulaciones, sacarle partido a esto
print(experimentHL.duration)

0:02:07.186522


In [16]:
def count_deadlocks(deadlock_time, final_time):
    is_deadlocked = np.logical_and(deadlock_time > 0, deadlock_time < (final_time - 5.0))
    return sum(is_deadlocked)

def extract_data(experiment):
    collisions = []
    deadlocks = []
    efficacy = []
    etas=[]
    taus=[]
    sms = []
    bas = []
    seeds = []
    for i, run in experiment.runs.items():
        world = run.world
        sm = np.unique([agent.behavior.safety_margin for agent in world.agents])
        tau = np.unique([agent.behavior.tau for agent in world.agents])
        eta = np.unique([agent.behavior.eta for agent in world.agents])
        ba = np.unique([agent.behavior.barrier_angle for agent in world.agents])
        bas += list(ba)
        taus += list(tau)
        etas += list(eta)
        sms += list(sm)
        seeds.append(run.seed)
        final_time = run.world.time
        deadlocks.append(count_deadlocks(run.deadlocks, final_time))
        collisions.append(len(run.collisions))
        efficacy.append(run.efficacy.mean())

    df = pd.DataFrame({
        'seeds': seeds,
        'safety_margin': sms,
        'eta': etas,
        'tau': taus,
        'deadlocks': deadlocks,
        'collisions': collisions,
        'barrier_angle': bas,
        'efficacy': efficacy})
    df['safe'] = (df.collisions == 0).astype(int)
    df['fluid'] = (df.deadlocks == 0).astype(int)
    df['ok'] = ((df.deadlocks == 0) & (df.collisions == 0)).astype(int)
    return df

In [17]:
dfHL = extract_data(experimentHL)
dfHL

,seeds,safety_margin,eta,tau,deadlocks,collisions,barrier_angle,efficacy,safe,fluid,ok
0,0,0.071519,0.548814,0.592845,0,0,1.570796,0.733661,1,1,1
1,1,0.072032,0.417022,0.997185,0,16,1.570796,0.689394,0,1,0
2,2,0.002593,0.435995,0.185082,0,61,1.570796,0.838109,0,1,0
3,3,0.070815,0.550798,0.070725,0,0,1.570796,0.781798,1,1,1
4,4,0.054723,0.967030,0.900621,0,31,1.570796,0.693256,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
995,995,0.098531,0.362305,0.942547,0,31,1.570796,0.644690,0,1,0
996,996,0.031503,0.197763,0.491709,0,38,1.570796,0.795346,0,1,0
997,997,0.090836,0.445604,0.426907,12,0,1.570796,0.649764,1,0,0
998,998,0.066310,0.956562,0.628222,0,0,1.570796,0.688350,1,1,1


In [18]:
dfHL.describe()

,seeds,safety_margin,eta,tau,deadlocks,collisions,barrier_angle,efficacy,safe,fluid,ok
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,499.500000,0.049989,0.487860,0.493628,1.294000,33.664000,1.570796,0.750868,0.395000,0.848000,0.29400
std,288.819436,0.028335,0.291594,0.283004,3.890077,71.267174,0.000000,0.059937,0.489095,0.359201,0.45582
min,0.000000,0.000008,0.001283,0.001765,0.000000,0.000000,1.570796,0.483364,0.000000,0.000000,0.00000
25%,249.750000,0.025906,0.228026,0.247726,0.000000,0.000000,1.570796,0.717527,0.000000,1.000000,0.00000
50%,499.500000,0.050396,0.478904,0.487291,0.000000,5.000000,1.570796,0.759020,0.000000,1.000000,0.00000
75%,749.250000,0.074366,0.744665,0.724543,0.000000,31.000000,1.570796,0.793319,1.000000,1.000000,1.00000
max,999.000000,0.099995,0.998444,0.999028,29.000000,544.000000,1.570796,0.859312,1.000000,1.000000,1.00000


In [ ]:
import functions as fu
dfHL['persistentEntropy'] = None
for i in tqdm(range(simulations)):
    run = experimentHL.runs[i]
    ps = run.poses
    maxd = [float(np.max(distance_matrix(X,X))) for X in ps[:,:,:2]]
    entropias=[]
    for j in range(time_instances):
        persistence = fu.calculaDiagramaPersistencia(ps,j,0,"rips")
        persistenciaL = fu.limitaDiagrama(persistence,maxd[j])
        entropias.append(fu.calculaEntropia(persistenciaL))
    dfHL.at[i, 'persistentEntropy'] = np.array(entropias)

 25%|██████████████████▉                                                        | 252/1000 [3:41:24<11:00:56, 53.02s/it]

## Neural network example

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Suponiendo que df es tu dataframe con las columnas 'A' y 'B'

# Convertir las series temporales a una matriz de numpy
X = []
for i in dfHL["persistentEntropy"]:
    X.append(i)
X = np.array(X)
y = np.array(dfHL['safe'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y compilar el modelo
model = Sequential([
    Input((X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step - accuracy: 0.5000 - loss: 1.2419 - val_accuracy: 0.5000 - val_loss: 4.6206
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5000 - loss: 4.6142 - val_accuracy: 0.5000 - val_loss: 3.6635
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5000 - loss: 3.6555 - val_accuracy: 0.5000 - val_loss: 1.4868
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5000 - loss: 1.4779 - val_accuracy: 0.5000 - val_loss: 1.2969
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5000 - loss: 1.2911 - val_accuracy: 0.5000 - val_loss: 2.1544
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5000 - loss: 2.1401 - val_accuracy: 0.5000 - val_loss: 1.9311
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5000 - loss: 1.9171 - val_accuracy: 0.5000 - val_loss: 1.1431
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5000 - loss: 1.1329 - val_accuracy: 0.5000 - val_loss

In [13]:
y_test

array([1, 0])

In [14]:
model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[0.49882734],
       [0.49741888]], dtype=float32)